# Characterizing and Normalizing Dataset Time Axis

This notebook covers our efforts to normalize the time axes of the CUPRAC dataset in order to move toward a universal time index. This is needed as multivariate statistical models such as XGBoost require that the same feature (peak) is in the same column (time) for each sample. We will do this by treating the sample signals as time series.

In [ ]:
# setup

from wine_analysis_hplc_uv import definitions
from wine_analysis_hplc_uv.db_methods import get_data, pivot_wine_data
import pandas as pd
import duckdb as db
import matplotlib.pyplot as plt

pd.options.mode.copy_on_write = True
pd.options.display.width = None
pd.options.display.max_colwidth = 50
pd.options.display.max_rows = 20
pd.options.display.max_columns = 15
pd.options.display.colheader_justify = "left"

In [ ]:
# get data


def fetch_dataset(con):
    get_data.get_wine_data(
        con, detection=("cuprac",), wavelength=(450,), varietal=("shiraz",)
    )
    df = pivot_wine_data.pivot_wine_data(con)
    return df


con = db.connect(definitions.DB_PATH)
df = fetch_dataset(con)
df.head()

## Sample 154

The sample used to explore the fundamentals of the time series is sample 154 - 2020 leeuwin estate shiraz art series, an Australian Shiraz from Margaret River, Western Australia. A Shiraz has been selected because they, at least anecdotally, have the highest peak intensity and signal complexity, meaning that patterns in the data should be easy to detect.

In [ ]:
# get 154
df154 = df.loc[:, pd.IndexSlice["154", :, ["mins", "value"]]]
display(df154.info())

ax = (
    df154.stack(["samplecode", "wine"])
    .reset_index()
    .plot.line(
        x="mins",
        y="value",
        title=df154.columns.get_level_values(1)[0],
        ylabel="abs (mAU)",
    )
)
plt.tight_layout()

## Conversion to DateTime

As we will be treating the data as time series, we should first convert the time axis to `datetime-like`, in this case `timedelta`.

In [ ]:
# set mins to timedelta with units 'minutes', drop 'i', set minutes to index

df154 = (
    df154.stack(["samplecode", "wine"])
    .assign(mins=lambda df: pd.to_timedelta(df.mins, unit="minutes"))
    .reset_index()
    .drop("i", axis=1)
    .set_index(["samplecode", "wine", "mins"])
    .unstack(["samplecode", "wine"])
    .reorder_levels(["samplecode", "wine", "vars"], axis=1)
    .pipe(lambda df: df if display(df) else df)  # display df
);

## Measuring Sampling Frequency 

The use of of sampling methods requires a method of gauging the frequency, and regularity of frequency of observation of each dataset. Sampling frequency here is defined as the number of observations per second $\frac{n \space \text{obs}}{m \space \text{seconds}}$. I am expecting the sampling frequency to equal 2.5Hz.

In [ ]:
(
    df154.pipe(lambda df: df if display(df) else df)  # display df
    .index.to_frame()
    .rename_axis("mins_index")
    .diff(1)
    .rename({"mins": "diff"}, axis=1)
    .reset_index()
    .pipe(lambda df: df if display(df["diff"].value_counts()) else df)
    .agg({"diff": "mode"})
    .assign(diff=lambda df: df["diff"].dt.total_seconds() * 1000)
    .assign(freq=lambda df: (1 / df["diff"]) * 1000)
    .rename({"diff": "diff (ms)", "freq": "freq (Hz)"}, axis=1)
)

So we can see that the sampling frequency is one observation per 400 milliseconds, or 2.5 Hz, and that at least for this sample, the frequency is consistant. Thus, no extrenuous resampling is necessary beyond compression.

## Determining Maximum Time Precision

An unfortunate side-effect of floating-point data types [@_d] is that for a given experimental variable observation, and depending on the numerical data type, there will be a higher number of digits stored in memory than the actual precision of the instrument. As one of my goals is to align all of my time series to one universal time axis, decimial digits beyond an identified level of precision can be treated as noise and discarded without further thought. Thus I need a method of identifying what an appropriate level of precision is. Agilent is not forthcoming with the rating of their DAD, so an internal analysis is required. In [determining_time_precision](./determining_time_precision.ipynb) I observed what effects changing the time scale had on the granularity of the data, and increased the time scale until I identified that a millisecond scale was the highest I could go without resulting in duplicates. A round-about way of approaching the problem, but an effective one.

There is question of what is the precision of the time points of my observations. For example, sample 154:

In [ ]:
df154.head()

the second time point of this sample is:

In [ ]:
obs = df154.index.to_frame().astype(str).iat[1, 0]
print(obs)
print("num sigfigs:", len(obs.split(".")[1]))

 Unfortunately even the 'raw' data in my database has a precision of sometimes 18 digits, which could not possibly be correct, and must be a symptom of float datatypes in Python. To settle this once and for all, I could either make a decision of what is the minimum time scale that retains unique values in the time column, or check a .UV file.

In [ ]:
import rainbow as rb
import os

filepath = os.path.join(definitions.LIB_DIR, "cuprac", "131.D")
obs = rb.read(filepath).get_file("DAD1.UV").xlabels[0]
print(obs)
print("num sigfigs:", len(str(obs).split(".")[1]))

Well I have been vindicated, as rainbow is also returning 18 significant figures. Thus the second approach is required - identify an appropriate level of granularity by testing several time scales and seeing when duplicate values appear. Observe the millisecond ('L') and second ('S') scales (refer to [offset alias](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) for the symbology):

In [ ]:
df154_ = df154.pipe(lambda df: df.set_index(pd.to_timedelta(df.index, unit="minutes")))
df154_

In [ ]:
len(df154_.index[df154_.index.round(freq="L").duplicated()])

In [ ]:
len(df154_.index[df154_.index.round(freq="S").duplicated()])

It appears that no duplicates are detected at the millisecond scale ('L') , however at the second ('S') scale, over half the observation points are now duplicates. Thus we will continue at the millisecond scale. To reiterate, the time axis should be converted to a `timedelta_range` and then rounded to "L", or millisecond scale, in the following manner:

First reset the index to float:

In [ ]:
df154 = (
    df154.pipe(lambda df: df.set_axis(df.index.total_seconds() / 60)).pipe(
        lambda df: df if display(df) else df
    )  # display df
)

Then convert to timedelta and round:

In [ ]:
df154 = (
    df154.pipe(lambda df: df.set_axis(pd.to_timedelta(df.index, unit="minutes")))
    .pipe(lambda df: df.set_axis(df.index.round("L")))
    .pipe(lambda df: df if display(df) else df)  # display df
);

This simple operation has been added to [SignalProcessor](src/wine_analysis_hplc_uv/signal_processing/mindex_signal_processing.py) as `.adjust_timescale()`.

## Identifying and Removing Scalar Offset

While determining the time precision, I noticed that once adjusted to a millisecond scale, it was clear that there was a scalar offset in sample 154 of 15 milliseconds at element zero. This is odd as we would expect the zeroth element to have time zero, as it is the first observation. First lets observe if there are other offsets, and what their magnitude might be:

In [ ]:
df154 = (
    df154.index.to_frame()
    .rename_axis("mins_index")
    .diff(1)
    .rename({"mins": "diff"}, axis=1)
    .reset_index()
    .assign(diff_mode=lambda df: df.loc[:, "diff"].mode())
    .assign(mins_index=lambda df: df.loc[:, "mins_index"] - df.loc[0, "mins_index"])
    .pipe(lambda df: df if display(df) is None else df)
)

So we can se that it is infact a scalar offset of exactly 400 milliseconds. Now the question is, will that work for every sample? Lets get the average observation to observation time difference across each signal and compare:


In [ ]:
"""
Get all the sample data from the db, select the time column, convert them to 
timedelta, round to milliseconds. Then per group, add a 'diff' and `mean_diff` column,
then return to tidy format.
"""


def fetch_all_samples(con):
    get_data.get_wine_data(con, detection=("cuprac",), wavelength=(450,))
    df = (
        pivot_wine_data.pivot_wine_data(con)
        .loc[:, pd.IndexSlice[:, :, ["mins"]]]  #
        .stack(["samplecode", "wine"])
        .assign(
            mins=lambda df: df.groupby(["samplecode", "wine"])["mins"]
            .transform(pd.to_timedelta, **dict(unit="minutes"))
            .round("L"),
            diff=lambda df: df.groupby(["samplecode", "wine"])["mins"].diff(),
            mean_diff=lambda df: df.groupby(["samplecode", "wine"])["mins"]
            .diff()
            .mean(),  # not sure why but agg'ing on the diff column results in NaN
        )
        .unstack(["samplecode", "wine"])
        .reorder_levels(["samplecode", "wine", "vars"], axis=1)
        .sort_index(level=0, axis=1, sort_remaining=True)
    )
    return df


adf = fetch_all_samples(con)
adf.head()

In [ ]:
# aggregate the mean_diff results for each group and plot
(
    adf.stack(["samplecode", "wine"])
    .groupby(["samplecode", "wine"])["mean_diff"]
    .first()
    .plot(xlabel="sample", ylabel="mean time value diff")
)

As we can see there is a consistant scalar observation per observation time difference of 400 milliseconds, which matches the expected frequency of 2.5Hz.

Now what about the first observation offset, what is the trend?

In [ ]:
(
    adf.stack(["samplecode", "wine"])
    .groupby(["samplecode", "wine"])["mins"]
    .first()
    .plot(style=".", title="first time value per sample", ylabel="time (mins)")
)
plt.tick_params(axis="x", bottom=False, labelbottom=False)
plt.tight_layout()

We can see without further analysis that there is a random spread of values, thus we can be confident in merely subtracting that value from the time column, aligning observation zero with time zero:

In [ ]:
adf = (
    adf.stack(["samplecode", "wine"])
    .assign(
        mins=lambda df: df.groupby(["samplecode", "wine"])["mins"].transform(
            lambda x: x - x.iloc[0]
        )
    )  # adjust time axis by initial value so they all start at 1
    .unstack(["samplecode", "wine"])
    .reorder_levels(["samplecode", "wine", "vars"], axis=1)
    .sort_index(level=0, axis=1, sort_remaining=True)
    .pipe(lambda df: df if display(df.head()) else df)
)

In [ ]:
(
    adf.stack(["samplecode", "wine"])
    .groupby(["samplecode", "wine"])["mins"]
    .first()
    .plot(
        style=".",
        title="first time value per sample, first value subtracted",
        ylabel="time (mins)",
    )
)
plt.tick_params(axis="x", bottom=False, labelbottom=False)
plt.tight_layout()

Ok, that's convincing enough for me. As of 2023-08-23 22:47:37 I am going to assume the full dataset follows the same pattern. In summary: all data time axes have a varying offset equal to the value of the first measurement. Subtracting the first value from the axis will align the data so that the first measurement is zero. The caveat is that the observation frequency must be the same for all samples.

A method for correcting the offset has been created [here](src/wine_analysis_hplc_uv/signal_processing/mindex_signal_processing.py) under `.correct_offset`

## Creation of Universal Time Axis

Now that the offset has been corrected for, and rounding to milliseconds, the sample on sample time columns are looking regular, and I suspect that we can now use 1 universal time column as an index. To determine whether this is true, we should compare all time columns and find any outliers. I will investigate this by treating each time element as a column and calculating the z-score for each row in that column.


In [ ]:
"""
Select 'mins' column, convert to float (seconds), transpose, forward and backfill
missing values to prepare for outlier detection
"""

adfT = (
    adf.stack(["samplecode", "wine"])["mins"]
    .unstack(["samplecode", "wine"])
    .apply(lambda x: x.dt.total_seconds())
    .T.ffill()
    .bfill()
)

In [ ]:
"""
Calculate mean difference rounded to 3 decimal places (because floats), find those who 
are not equal to zero, then the total sum of True values
"""

import numpy as np

mask = (adfT.apply(lambda x: np.round(x - x.mean(), 3)) != 0).sum().sum()
mask

Subtracting the mean of each column from its elements should act as a outlier detector, as we are expecting the values to be either all equal, or not. As we can see, useing the condition `!=0` results in a boolean frame, and calling `.sum().sum()` will calculate the total number of elements which are not equal to zero. As we can see, that number is zero, thus all the time columns are now equal, and we can use a universal time column, rather than a inter-sample column.

## Conclusion

Observing both sample 154 and the overall dataset has enabled me to investigate treating the signals as time series, normalization methods, and moving towards a universal time axis. Specifically, we found that `pd.timedelta` was an appropriate time series datatype. Firstly we observed that in sample 154 there was a sampling frequency of 2.5Hz, or one observation per 400 milliseconds and then later on found this consistant across the whole CUPRAC dataset. Initally the time axes of each sample looked unreconcilable, but after rounding to a millisecond scale and subtracting a scalar offset, we proved that there was infact a universal time scale that can be used for all samples. The only caveat is that the samples need to be recorded at the same frequency. That being said, simple resampling would rectify those differences. Finally, methods to move to the universal time axis (index) have been created in `m_index_signal_processing`.